## Libraries

In [39]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from sklearn.datasets import load_boston
import pandas as pd
from pathlib import Path

In [40]:
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [41]:
%run ..//Libraries//Mape_extendido.ipynb 

## Load data

In [42]:
rng = check_random_state(0)
boston = load_boston()
perm = rng.permutation(boston.target.size)
boston.data = boston.data[perm]
boston.target = boston.target[perm]
# 
Y = boston.target
X = boston.data

In [43]:
#split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 5)

#normalize
scaler = MaxAbsScaler().fit(x_train)
X_train = scaler.transform(x_train); X_test = scaler.transform(x_test)

scaler1 = MaxAbsScaler().fit(y_train.reshape(-1, 1))
Y_train = scaler1.transform(y_train.reshape(-1, 1)).reshape(-1)
Y_test = scaler1.transform(y_test.reshape(-1, 1)).reshape(-1)

## Optimization function

In [44]:
def opt_bas(C, epsilon, lamda, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'epsilon' : epsilon, 
        'lamda' : lamda,
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR_cvxopt_mapext(**hyperparameters).fit(X_train, Y_train)

    predict = model.predict(X_test)
    
    # rescale
    y_pred = scaler1.inverse_transform(predict.reshape(-1, 1)).reshape(-1)
    
    # get score
    mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
    
    return -mape

## Run optimization

In [45]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [46]:
# Bounded region of parameter space
pbounds = {'C': (0.01, 10), 'epsilon': (0.001, 10), 'lamda': (0.001, 0.5), 'gamma': (0.01, 2)}

# Domain reduction function
# bounds_transformer = SequentialDomainReductionTransformer()

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
#                                  bounds_transformer=bounds_transformer
                                )

# keep data
log_path = Path().resolve() / "Logs" / "onepoint.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [47]:
optimizer.probe(
    params={"C": 0.1, "epsilon": 0.01, "lamda": 0.2, "gamma": 0.3},
    lazy=True,
)

In [48]:
optimizer.maximize(init_points=1, n_iter=200)

In [ ]:
# {"target": -9.311010260315642, "params": {"C": 1.9899589004517189, "epsilon": 8.715423817392129, "gamma": 0.050855817169792, "lamda": 0.2949374506115638}, "datetime": {"datetime": "2021-06-17 11:21:27", "elapsed": 34.208827, "delta": 0.853526}}